In [1]:
#Fairness Metric Trend (Before vs After Mitigation)

#Description:
#A line chart with SPD (Y-axis) over inference batches (X-axis).

#·       Red curve: Baseline system (shows sharp spikes to −0.17)

#·       Blue curve: After mitigation (steady around −0.08)
#Add a shaded band representing the acceptable fairness threshold (±0.10).

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Fairness SPD trend (Baseline vs After Defense)
# This cell reads the summary CSV, synthesizes batch-level SPD series for illustration,
# and plots the two curves with a shaded acceptable band (±0.10).
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read CSV (fixed path)
csv_path = os.path.join('data', 'fairness_metrics.csv')
print(f'Reading CSV from: {csv_path}')
df = pd.read_csv(csv_path)
print(df.head())

# Extract SPD baseline and after values
spd_row = df[df['Metric'].str.contains('Statistical Parity', case=False, na=False)]
if spd_row.empty:
    raise RuntimeError('Could not find Statistical Parity Difference row in CSV')

baseline_spd = float(spd_row['Baseline (Before Defense)'].values[0])
after_spd = float(spd_row['After Defense Framework'].values[0])

# Create synthetic batch-level series (illustrative only)
np.random.seed(42)
n_batches = 60
batches = np.arange(1, n_batches + 1)

# Baseline: generally worse, with occasional sharp spikes reaching baseline_spd
baseline_mean = baseline_spd - 0.08  # make the "normal" baseline slightly worse than the spike value
baseline = np.random.normal(loc=baseline_mean, scale=0.03, size=n_batches)
# insert a few sharp spikes (set their value near the reported baseline_spd)
spike_indices = np.random.choice(n_batches, size=6, replace=False)
for idx in spike_indices:
    baseline[idx] = baseline_spd + np.random.normal(scale=0.01)

# After mitigation: steady around after_spd with low variance
after = np.random.normal(loc=after_spd, scale=0.01, size=n_batches)

# Plot
# Try to use seaborn whitegrid if available; otherwise fall back to matplotlib styles
try:
    import seaborn as sns
    sns.set_theme(style='whitegrid')
except Exception:
    try:
        plt.style.use('seaborn')
    except Exception:
        plt.style.use('default')

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(batches, baseline, color='red', marker='o', linestyle='-', label='Baseline')
ax.plot(batches, after, color='blue', marker='o', linestyle='-', label='After Defense')

# Shaded acceptable fairness threshold band ±0.10
threshold = 0.10
ax.axhspan(-threshold, threshold, color='green', alpha=0.12, label='Acceptable threshold (±0.10)')

ax.set_xlabel('Inference Batch')
ax.set_ylabel('Statistical Parity Difference (SPD)')
ax.set_title('Fairness Metric Trend (SPD) — Baseline vs After Defense')
ax.legend(loc='upper right')

# Mark the spike indices for visibility
ax.scatter(batches[spike_indices], baseline[spike_indices], color='darkred', s=80, edgecolor='k', zorder=5, label='Baseline spikes')

# Save output
out_dir = 'out'
os.makedirs(out_dir, exist_ok=True)
out_path = os.path.join(out_dir, 'fairness_spd_trend.png')
fig.tight_layout()
fig.savefig(out_path, dpi=150)

print(f'Plot saved to: {out_path}')
plt.show()

Reading CSV from: data\fairness_metrics.csv
                                       Metric  \
0         Statistical Parity Difference (SPD)   
1             Equalized Odds Difference (EOD)   
2                                   Model AUC   
3  Fairness Drift Rate (per 1000 predictions)   
4          Recovery Time (MTTR) after anomaly   

                                         Description  \
0  Measures difference in positive prediction rat...   
1  Average absolute difference of TPR/FPR between...   
2            Overall model discrimination capability   
3  Count of drift alerts triggered by fairness mo...   
4  Time (in minutes) for automatic rollback and r...   

  Baseline (Before Defense) After Defense Framework  
0                     -0.17                   -0.08  
1                      0.21                    0.09  
2                      0.88                    0.87  
3                        18                       3  
4                    12 min                   4 min  


OSError: 'seaborn-whitegrid' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)